In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from DomainPrediction import BaseProtein
from DomainPrediction.esm.esm2 import ESM2
from DomainPrediction.eval import metrics
from DomainPrediction.utils import helper

In [ ]:
protein = BaseProtein(file='../../Data/GxpS_ATC_AF.pdb')
T = [i for i in range(538,608)] ## 539-608

In [ ]:
protein.get_residues(T) ## T domain

In [ ]:
masked_query = ''.join(['<mask>' if i in T else protein.sequence[i] for i in range(len(protein.sequence))])

In [ ]:
masked_query

In [ ]:
# model_path = '/data/users/kgeorge/workspace/esm2/checkpoints/esm2_t30_150M_UR50D.pt'
model_path = '/data/users/kgeorge/workspace/esm2/checkpoints/esm2_t6_8M_UR50D.pt'
esm2 = ESM2(model_path = model_path, device='gpu')

In [ ]:
prob = esm2.get_prob(sequence=masked_query)

In [ ]:
perplexity = metrics.compute_perplexity(esm2, protein.sequence)

In [ ]:
fasta_path = '../../Data/esm_experiments/gen_1000/esm_inp_seq_1000.fasta'
meta_path = '../../Data/esm_experiments/gen_1000/pdbs_GxpS_ATC'

In [ ]:
records = helper.read_fasta(fasta_path)
for rec in records:
    perplexity = metrics.compute_perplexity(esm2, str(rec.seq))
    meta_file = os.path.join(meta_path, rec.id + '.meta.npz')
    helper.update_metadata(meta_file, 'esm2_8M_perplexity', perplexity, force=False)